In [1]:
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 29.9 MB/s eta 0:00:00


In [1]:
import re
import os
import requests
from typing import Dict, Optional
import numpy as np

try:
    from tqdm import tqdm
    _has_tqdm = True
except ImportError:
    _has_tqdm = False


import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
# или попробовать:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score, f1_score, jaccard_score
import warnings
import os
import random
import inspect
from typing import List, Tuple, Optional, Union

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch import nn
from tqdm import tqdm

import monai
from monai.networks.nets import SwinUNETR
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from scipy import ndimage as ndi

<frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.


In [3]:
import os
import re
import zipfile
import gdown
import numpy as np

def download_and_unpack_gdrive_gdown(url_or_id, dest_folder='.', out_filename=None,
                                     save_npy=False, quiet=False):
    """
    Скачивает файл с Google Drive через gdown и распаковывает
    """
    os.makedirs(dest_folder, exist_ok=True)

    # если передали только id — соберём URL
    if re.fullmatch(r"[A-Za-z0-9_-]{10,}", url_or_id):
        url = f"https://drive.google.com/uc?id={url_or_id}"
    else:
        url = url_or_id

    # имя выходного файла
    if out_filename is None:
        # попытка взять имя из URL, иначе дать дефолт
        if "id=" in url:
            out_filename = f"downloaded_{url.split('id=')[-1]}.bin"
        else:
            out_filename = os.path.basename(url) or "downloaded_file.bin"

    out_path = os.path.join(dest_folder, out_filename)

    # Скачиваем
    if not os.path.exists(out_path):
        gdown.download(url, out_path, quiet=quiet)

In [4]:
file_id = "1VmU4q1ySfRlrRcIawWBmqXPJ-fcAMnxO"
url = f"https://drive.google.com/uc?id={file_id}"
download_and_unpack_gdrive_gdown(url_or_id=url,
                                           dest_folder="/kaggle/working",
                                           out_filename="checkpoint.pth",
                                           quiet=False)

file_id = "1NOUwAFtU_1KIlf8mQtB5xwSY-3NtYnDw"
url = f"https://drive.google.com/uc?id={file_id}"
download_and_unpack_gdrive_gdown(url_or_id=url,
                                           dest_folder="/kaggle/working",
                                           out_filename="seis_train.npz",
                                           quiet=False)

out_path = os.path.join("/kaggle/working", "seis_train.npz")

file_id = "1QHE1gn7B8Zt99Cw1KDwyfNv6wqhM5h49"
url = f"https://drive.google.com/uc?id={file_id}"
fault_val = download_and_unpack_gdrive_gdown(url_or_id=url,
                                           dest_folder="/kaggle/working",
                                           out_filename="fault_train.npz",
                                           quiet=False)
file_id = "1vVOkYAZkq08CtWosLd27Py4quvAaTjOa"
url = f"https://drive.google.com/uc?id={file_id}"
fault_val = download_and_unpack_gdrive_gdown(url_or_id=url,
                                           dest_folder="/kaggle/working",
                                           out_filename="seis_val.npz",
                                           quiet=False)
file_id = "1YIeB6J69RUozpKwWa84m8ZXgpaQMWmP7"
url = f"https://drive.google.com/uc?id={file_id}"
fault_val = download_and_unpack_gdrive_gdown(url_or_id=url,
                                           dest_folder="/kaggle/working",
                                           out_filename="fault_val.npz",
                                           quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1VmU4q1ySfRlrRcIawWBmqXPJ-fcAMnxO
From (redirected): https://drive.google.com/uc?id=1VmU4q1ySfRlrRcIawWBmqXPJ-fcAMnxO&confirm=t&uuid=dfc9d4bf-8a19-4aed-8b9f-2376550c4cf6
To: /kaggle/working/checkpoint.pth
100%|██████████| 731M/731M [00:12<00:00, 58.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1NOUwAFtU_1KIlf8mQtB5xwSY-3NtYnDw
From (redirected): https://drive.google.com/uc?id=1NOUwAFtU_1KIlf8mQtB5xwSY-3NtYnDw&confirm=t&uuid=82e64845-accc-4729-b57c-c97bdf5fecbd
To: /kaggle/working/seis_train.npz
100%|██████████| 7.72G/7.72G [01:41<00:00, 76.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1QHE1gn7B8Zt99Cw1KDwyfNv6wqhM5h49
From (redirected): https://drive.google.com/uc?id=1QHE1gn7B8Zt99Cw1KDwyfNv6wqhM5h49&confirm=t&uuid=0c34fff1-c73f-48be-9662-043da76b7cff
To: /kaggle/working/fault_train.npz
100%|██████████| 18.4M/18.4M [00:00<00:00, 53.8MB/s]
Downloading...
From (original): https:

In [2]:
from typing import Union, Sequence, Tuple, Optional, Dict, Iterable
import random
import os
import re
import numpy as np
import torch
from torch.utils.data import Dataset

class SeisFaultDataset:
    """
    Датасет для пар (сейсмика, fault) в виде 3D numpy-массивов.

    Поддерживаемые источники:
    - путь к .npz файлу (будет открыт через np.load(..., mmap_mode='r'))
    - dict-like mapping имя -> np.ndarray

    Парирование:
    - если pairs не переданы, автоматически берётся отсортированное пересечение ключей
      между seismic и fault источниками. Если пересечение пусто — бросается исключение.

    Основные опции:
    - target_shape: размер кропа (z,y,x)
    - random_crop: True — случайные кропы, False — кропы с нулевого/центрального начала
    - seed: для детерминированности
    - normalize_seis: применять z-score нормализацию к сейсмике
    """

    def __init__(
        self,
        seis_source: Union[str, Dict[str, np.ndarray]],
        fault_source: Union[str, Dict[str, np.ndarray]],
        pairs: Optional[Sequence[Tuple[str, str]]] = None,
        target_shape: Tuple[int, int, int] = (128, 128, 128),
        random_crop: bool = True,
        seed: Optional[int] = None,
        normalize_seis: bool = True,
    ):
        # параметры
        self.target_shape = tuple(int(x) for x in target_shape)
        if len(self.target_shape) != 3:
            raise ValueError("target_shape должен быть кортежем длины 3")
        self.random_crop = bool(random_crop)
        self.normalize_seis = bool(normalize_seis)
        self._rng = random.Random(seed)

        # загрузка источников (path -> npz mmap или оставляем dict)
        self._seis_npz = None
        self._fault_npz = None

        if isinstance(seis_source, str):
            if not os.path.exists(seis_source):
                raise FileNotFoundError(seis_source)
            self._seis_npz = np.load(seis_source, mmap_mode='r')
            seis_source = self._seis_npz

        if isinstance(fault_source, str):
            if not os.path.exists(fault_source):
                raise FileNotFoundError(fault_source)
            self._fault_npz = np.load(fault_source, mmap_mode='r')
            fault_source = self._fault_npz

        # ожидаем mapping name->ndarray
        self.seis_source = seis_source
        self.fault_source = fault_source

        # парами: либо явно, либо пересечение ключей
        if pairs is not None:
            self.pairs = list(pairs)
        else:
            seis_keys = self._keys_from_source(self.seis_source)
            fault_keys = self._keys_from_source(self.fault_source)

            # пары (имя, имя)
            self.pairs = [(seis, fault) for seis, fault in zip(seis_keys, fault_keys)]

        if len(self.pairs) == 0:
            raise ValueError("No pairs available")

    # ---- Вспомогательные методы ----

    def _keys_from_source(self, src):
        """Возвращает список ключей для mapping-источника или .files для npz."""
        if hasattr(src, 'files'):
            return list(src.files)
        if isinstance(src, dict):
            return list(src.keys())
        # obj, у которого можно взять ключи через итерацию
        try:
            return list(src.keys())
        except Exception:
            raise ValueError(f"Unsupported source type: {type(src)}")

    def _get_array(self, src, key):
        """Простая загрузка массива; ожидаем numpy.ndarray 3D."""
        arr = src[key]
        if not isinstance(arr, np.ndarray):
            # попытка конвертации (обычно не нужна)
            arr = np.asarray(arr)
        if arr.ndim != 3:
            raise ValueError(f"Ключ {key}: ожидается 3D массив, получено {arr.ndim}D")
        return arr

    def _compute_crop_start(self, shape: Tuple[int, int, int]) -> Tuple[int, int, int]:
        """Вычислить стартовые индексы для кропа (случайно или ноль/центр)."""
        starts = []
        for i in range(3):
            max_start = shape[i] - self.target_shape[i]
            if max_start <= 0:
                starts.append(0)
            else:
                if self.random_crop:
                    starts.append(self._rng.randint(0, max_start))
                else:
                    # по умолчанию — центрированный кроп если есть запас
                    starts.append(max_start // 2)
        return tuple(starts)

    def _zscore_normalize(self, vol: np.ndarray) -> np.ndarray:
        """Z-score нормализация (для сейсмики)."""
        v = vol.astype(np.float32, copy=False)
        mu = v.mean()
        sigma = v.std()
        eps = 1e-8
        return (v - mu) / (sigma + eps)


    def __len__(self) -> int:
        return len(self.pairs)

    def __getitem__(self, idx: int):
        """Вернёт (seis_crop, fault_crop, meta)."""
        if idx < 0:
            idx = len(self) + idx
        seis_key, fault_key = self.pairs[idx]

        seis = self._get_array(self.seis_source, seis_key)
        fault = self._get_array(self.fault_source, fault_key)

        # берем кроп по форме seismic-а
        starts = self._compute_crop_start(seis.shape)
        slices = tuple(slice(st, st + ts) for st, ts in zip(starts, self.target_shape))

        seis_crop = seis[slices]
        fault_crop = fault[slices]

        if self.normalize_seis:
            seis_crop = self._zscore_normalize(seis_crop)

        meta = {
            "seis_key": seis_key,
            "fault_key": fault_key,
            "crop_start": starts,
            "original_shapes": (seis.shape, fault.shape),
        }
        return torch.unsqueeze(torch.Tensor(seis_crop), 0), torch.unsqueeze(torch.Tensor(fault_crop), 0), meta

    def close(self):
        """Закрыть mmap .npz если он был открыт."""
        if self._seis_npz is not None:
            try:
                self._seis_npz.close()
            except Exception:
                pass
            self._seis_npz = None
        if self._fault_npz is not None:
            try:
                self._fault_npz.close()
            except Exception:
                pass
            self._fault_npz = None

    def __del__(self):
        self.close()

In [3]:
train_ds = SeisFaultDataset("/kaggle/working/seis_train.npz", "/kaggle/working/fault_train.npz",
                      target_shape=(128,128,128), seed=42)

val_ds = SeisFaultDataset("/kaggle/working/seis_val.npz", "/kaggle/working/fault_val.npz",
                      target_shape=(128,128,128), seed=42)

In [4]:
import os
import re
import zipfile
import gdown
import numpy as np

def download_and_unpack_gdrive_gdown(url_or_id, dest_folder='.', out_filename=None,
                                     save_npy=False, quiet=False):
    """
    Скачивает файл с Google Drive через gdown и распаковывает:
      - если .npz -> загружает и (по умолчанию) сохраняет каждый массив как .npy
      - если .zip -> распаковывает архив в dest_folder
      - иначе пытается определить тип и вернуть путь к скачанному файлу

    Аргументы:
      url_or_id: полный URL Google Drive или только file_id
      dest_folder: папка для сохранения/распаковки
      out_filename: имя сохраняемого файла (опционально)
      save_npy: сохранять ли отдельные .npy файлы при распаковке .npz
      quiet: флаг для gdown.download (False -> подробный вывод)
    Возвращает:
      dict с загруженными массивами (для .npz) или информацию о распаковке (для .zip)
    """
    os.makedirs(dest_folder, exist_ok=True)

    # если передали только id — соберём URL
    if re.fullmatch(r"[A-Za-z0-9_-]{10,}", url_or_id):
        url = f"https://drive.google.com/uc?id={url_or_id}"
    else:
        url = url_or_id

    # имя выходного файла
    if out_filename is None:
        # попытка взять имя из URL, иначе дать дефолт
        if "id=" in url:
            out_filename = f"downloaded_{url.split('id=')[-1]}.bin"
        else:
            out_filename = os.path.basename(url) or "downloaded_file.bin"

    out_path = os.path.join(dest_folder, out_filename)

    # Скачиваем
    if not os.path.exists(out_path):
        gdown.download(url, out_path, quiet=quiet)

In [5]:

file_id = "1gt1ig-To2DcbKmvFQL9UCKW8zMvgJy9f"
url = f"https://drive.google.com/uc?id={file_id}"
download_and_unpack_gdrive_gdown(url_or_id=url,
                                           dest_folder="/kaggle/working",
                                           out_filename="client_secret.json",
                                           quiet=False)

out_path = os.path.join("/content", "client_secret.json")

In [6]:

file_id = "1ksI9_TpSJ8tAZp88MFV3j4z-paDuaJbY"
url = f"https://drive.google.com/uc?id={file_id}"
download_and_unpack_gdrive_gdown(url_or_id=url,
                                           dest_folder="/kaggle/working",
                                           out_filename="token.json",
                                           quiet=False)

out_path = os.path.join("/content", "token.json")

In [7]:
try:
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaFileUpload
    from google.auth.transport.requests import Request
    from googleapiclient.errors import HttpError
except Exception:
    # в ноутбуке команда с !pip обычно работает — оставляем как есть
    try:
        print("[gdrive] google libs not found — attempting to install (may take a moment)...")
        get_ipython()  # if in notebook
        !pip install --quiet google-api-python-client google-auth google-auth-oauthlib
        from googleapiclient.discovery import build
        from googleapiclient.http import MediaFileUpload
        from google.auth.transport.requests import Request
        from googleapiclient.errors import HttpError
    except Exception as e:
        print("[gdrive] Failed to import google libs after install attempt:", e)
        # we will still define placeholders below

def setup_gdrive_from_token(token_path: str, folder_name: str):
    """
    token.json должен быть pickled credentials (pickle.dump(creds))
    Возвращает (service, folder_id) или (None, None)
    """
    if not os.path.exists(token_path):
        print(f"[gdrive] token.json not found at {token_path}. Upload token.json to enable uploads.")
        return None, None
    try:
        with open(token_path, "rb") as f:
            creds = pickle.load(f)
    except Exception as e:
        print("[gdrive] Failed to load token.json:", e)
        return None, None

    try:
        if creds and getattr(creds, "expired", False) and getattr(creds, "refresh_token", None):
            try:
                creds.refresh(Request())
                with open(token_path, "wb") as f:
                    pickle.dump(creds, f)
                print("[gdrive] Token refreshed and saved.")
            except Exception as e:
                print("[gdrive] Token refresh failed:", e)
    except Exception as e:
        print("[gdrive] Token refresh attempt exception:", e)

    try:
        service = build("drive", "v3", credentials=creds, cache_discovery=False)
    except Exception as e:
        print("[gdrive] Failed to build Drive service:", e)
        return None, None

    try:
        q = f"name = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
        res = service.files().list(q=q, fields="files(id, name)").execute()
        files = res.get("files", [])
        if files:
            folder_id = files[0]["id"]
            print(f"[gdrive] Found folder '{folder_name}' id={folder_id}")
        else:
            meta = {"name": folder_name, "mimeType": "application/vnd.google-apps.folder"}
            created = service.files().create(body=meta, fields="id").execute()
            folder_id = created.get("id")
            print(f"[gdrive] Created folder '{folder_name}' id={folder_id}")
        print("Folder link: https://drive.google.com/drive/folders/" + folder_id)
        return service, folder_id
    except Exception as e:
        print("[gdrive] Failed to find/create folder:", e)
        return service, None

def try_upload_file(service, folder_id, local_path):
    if not os.path.exists(local_path):
        print(f"[upload] SKIP — file not found: {local_path}")
        return False
    if service is None or folder_id is None:
        print(f"[upload] SKIP — Drive not configured (service set? {service is not None}, folder_id set? {folder_id is not None})")
        return False
    try:
        media = MediaFileUpload(local_path, resumable=True)
        meta = {"name": os.path.basename(local_path), "parents": [folder_id]}
        req = service.files().create(body=meta, media_body=media, fields="id")
        resp = req.execute()
        fid = resp.get("id")
        print(f"[upload] SUCCESS: {local_path} -> fileId={fid}")
        return True
    except HttpError as e:
        try:
            content = e.content.decode() if hasattr(e, "content") else str(e)
        except Exception:
            content = str(e)
        print(f"[upload] HttpError while uploading {local_path}: {content}")
        return False
    except Exception as e:
        print(f"[upload] Exception while uploading {local_path}: {e}")
        return False

# Путь к client_secret.json (если вы хотите пробовать interactive flow в Kaggle)
CLIENT_SECRETS_PATH = "/content/client_secret.json"  # если используете интерактивный flow
# Путь к token.json (предпочтительно положите сюда из локальной машины после генерации)
TOKEN_PATH = "/content/token.json"
# Имя папки в вашем Google Drive куда будут загружаться файлы
GDRIVE_FOLDER_NAME = "kaggle_swinunetr_experiments"

In [8]:
from monai.inferers import sliding_window_inference

ROI_SIZE = (128,128,128)
OVERLAP = 0.25
SW_BATCH_SIZE = 1

def train_epoch(model, dataloader, optimizer, loss_fn, epoch, scaler, main_device, accumulation_steps=1):
    model.train()
    running_loss = 0.0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Train E{epoch}")
    device0 = main_device

    for i, batch in pbar:
        if len(batch) == 3:
            imgs, masks, metas = batch
        else:
            imgs, masks = batch

        # Dataset даёт (B, C, D, H, W) — не нужно добавлять оси
        imgs = imgs.float().to(device0, non_blocking=True).contiguous()
        masks = masks.float().to(device0, non_blocking=True).contiguous()

        # защита: уверимся, что формы корректны
        assert imgs.ndim == 5, f"imgs must be 5D (B,C,D,H,W), got {imgs.shape}"
        assert masks.ndim == 5, f"masks must be 5D (B,C,D,H,W), got {masks.shape}"

        optimizer.zero_grad()
        #with torch.amp.autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
        outputs = model(imgs)
        loss = loss_fn(outputs, masks) / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        running_loss += float(loss.item() * accumulation_steps)
        pbar.set_postfix(loss=running_loss / (i + 1))

        del outputs, loss
        torch.cuda.empty_cache()

    return running_loss / max(1, len(dataloader))

def validate(model, dataloader, main_device):
    model.eval()
    val_loss = 0.0
    TP = FP = FN = 0
    predictor = unwrap_model(model)
    device0 = main_device

    with torch.no_grad():
        pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Val")
        for i, batch in pbar:
            if len(batch) == 3:
                imgs, masks, metas = batch
            else:
                imgs, masks = batch

            # не добавляем лишнюю ось
            inputs = imgs.float().to(device0, non_blocking=True)
            masks = masks.float().to(device0, non_blocking=True)

            assert inputs.ndim == 5, f"inputs must be 5D (B,C,D,H,W), got {inputs.shape}"
            assert masks.ndim == 5, f"masks must be 5D (B,C,D,H,W), got {masks.shape}"

            logits = sliding_window_inference(
                inputs=inputs,
                roi_size=ROI_SIZE,
                sw_batch_size=SW_BATCH_SIZE,
                predictor=predictor,
                overlap=OVERLAP
            )

            val_loss += float(loss_fn(logits, masks).item())

            probs = torch.sigmoid(logits)
            preds = (probs >= 0.5).float()

            preds_flat = preds.view(-1)
            masks_flat = masks.view(-1)

            TP += int(((preds_flat == 1.0) & (masks_flat == 1.0)).sum().item())
            FP += int(((preds_flat == 1.0) & (masks_flat == 0.0)).sum().item())
            FN += int(((preds_flat == 0.0) & (masks_flat == 1.0)).sum().item())

    val_loss = val_loss / max(1, len(dataloader))
    prec = TP / (TP + FP) if (TP + FP) > 0 else float("nan")
    rec = TP / (TP + FN) if (TP + FN) > 0 else float("nan")
    denom = 2 * TP + FP + FN
    val_f1 = 2.0 * TP / denom if denom > 0 else float("nan")
    return val_loss, val_f1, prec, rec


In [9]:
import torch
import os

def _strip_module_prefix(sd):
    new = {}
    for k, v in sd.items():
        if k.startswith("module."):
            new[k[len("module."):]] = v
        else:
            new[k] = v
    return new

def _add_module_prefix(sd):
    return {"module." + k: v for k, v in sd.items()}

def _intersect_state_dicts(src_sd, target_sd):
    """Вернуть state_dict, содержащий только те пары из src_sd, которые есть в target_sd и совпадают по форме."""
    filtered = {}
    target_items = target_sd.items()
    target_shapes = {k: v.shape for k, v in target_items}
    for k, v in src_sd.items():
        if k in target_shapes and getattr(v, "shape", None) == target_shapes[k]:
            filtered[k] = v
    return filtered

def load_checkpoint(
    ckpt_path,
    model,
    optimizer=None,
    scheduler=None,
    scaler=None,
    device=torch.device("cpu"),
    load_optimizer=True,
    load_scheduler=True,
    load_scaler=True,
    strict=True,
):
    """
    Загружает чекпойнт, делая безопасные проверки:
      - корректно обрабатывает module. префикс и DataParallel
      - при strict=False пытается частично загрузить пересекающиеся слои
      - аккуратно обрабатывает ошибки optimizer/scheduler/scaler (не падает)
    Возвращает: (ckpt_dict, next_epoch, metrics, info)
      - info содержит diagnostic поля: missing_keys, unexpected_keys, loaded_params_count и др.
    """
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    # безопасный map_location
    ckpt = torch.load(ckpt_path, map_location=device)

    # model_state может называться по-разному
    model_state = ckpt.get("model_state_dict", ckpt.get("state_dict", None))
    if model_state is None:
        raise RuntimeError("Checkpoint does not contain 'model_state_dict' or 'state_dict'")

    # unwrap модель если DataParallel -> работаем с model.module
    is_dp_model = isinstance(model, torch.nn.DataParallel)
    model_to_load = model.module if is_dp_model else model
    target_sd = model_to_load.state_dict()

    info = {"missing_keys": [], "unexpected_keys": [], "loaded_param_count": 0, "notes": []}

    # Попробуем разные варианты (префикс/без префикса)
    src_sd = model_state
    # если ключи объекта — не tensor'ы (редко), предполагаем что это корректный dict
    # Проверим префиксы
    src_keys_sample = list(src_sd.keys())[:5]
    target_keys_sample = list(target_sd.keys())[:5]

    # Если src имеет module. а target нет -> снять префикс
    if any(k.startswith("module.") for k in src_sd.keys()) and not any(k.startswith("module.") for k in target_sd.keys()):
        src_sd = _strip_module_prefix(src_sd)
        info["notes"].append("Stripped 'module.' prefix from checkpoint keys.")
    # Если src не имеет module. а target ожидает module. -> добавить префикс
    elif not any(k.startswith("module.") for k in src_sd.keys()) and any(k.startswith("module.") for k in target_sd.keys()):
        src_sd = _add_module_prefix(src_sd)
        info["notes"].append("Added 'module.' prefix to checkpoint keys to match DataParallel model.")

    # Попытка загрузки штатно
    try:
        res = model_to_load.load_state_dict(src_sd, strict=strict)
        # res is NamedTuple(missing_keys, unexpected_keys) when strict=False (or empty if strict=True and succeeded)
        missing = getattr(res, "missing_keys", []) or []
        unexpected = getattr(res, "unexpected_keys", []) or []
        info["missing_keys"] = missing
        info["unexpected_keys"] = unexpected
        info["loaded_param_count"] = sum(1 for k in src_sd.keys() if k in target_sd and src_sd[k].shape == target_sd[k].shape)
        print(f"[load] Model weights loaded (strict={strict}). Missing: {len(missing)}, Unexpected: {len(unexpected)}.")
    except RuntimeError as e:
        # Если strict=True — будем пробовать частичную загрузку при возможности
        print("[load] load_state_dict failed:", e)
        if strict:
            print("[load] strict=True и загрузка провалилась. Попробуйте вызвать с strict=False или отладить mismatch ключей.")
            raise
        else:
            # Попытка частичной загрузки: взять пересекающиеся параметры по имени и форме
            filtered = _intersect_state_dicts(src_sd, target_sd)
            if len(filtered) == 0:
                raise RuntimeError("[load] Частичная загрузка невозможна: нет совпадающих параметров по имени/форме.")
            missing_keys = [k for k in target_sd.keys() if k not in filtered.keys()]
            unexpected_keys = [k for k in filtered.keys() if k not in target_sd.keys()]
            model_to_load.load_state_dict(filtered, strict=False)
            info["missing_keys"] = missing_keys
            info["unexpected_keys"] = unexpected_keys
            info["loaded_param_count"] = len(filtered)
            info["notes"].append("Partial load: loaded only intersecting keys (name+shape).")

    # Попробуем загрузить optimizer/scheduler/scaler если нужно (и если они есть в чекпойнте)
    if optimizer is not None and load_optimizer:
        opt_state = ckpt.get("optimizer_state_dict", None)
        if opt_state is not None:
            try:
                optimizer.load_state_dict(opt_state)
                print("[load] Optimizer state loaded.")
            except Exception as e:
                print("[load] Не удалось загрузить optimizer state:", e)
                info["notes"].append("Optimizer load failed; keep current optimizer state. To avoid, create optimizer with same param groups or set load_optimizer=False.")
    if scheduler is not None and load_scheduler:
        sch_state = ckpt.get("scheduler_state_dict", None)
        if sch_state is not None:
            try:
                scheduler.load_state_dict(sch_state)
                print("[load] Scheduler state loaded.")
            except Exception as e:
                print("[load] Не удалось загрузить scheduler state:", e)
                info["notes"].append("Scheduler load failed; keep current scheduler state.")
    if scaler is not None and load_scaler:
        # в чекпойнте иногда называется 'scaler_state_dict' или 'amp_scaler'
        sc_state = ckpt.get("scaler_state_dict", ckpt.get("scaler", ckpt.get("amp_scaler", None)))
        if sc_state is not None:
            try:
                scaler.load_state_dict(sc_state)
                print("[load] GradScaler state loaded.")
            except Exception as e:
                print("[load] Не удалось загрузить scaler state:", e)
                info["notes"].append("GradScaler load failed; keep current scaler state.")

    # вернём полезные метаданные
    epoch = ckpt.get("epoch", None)
    next_epoch = (epoch + 1) if epoch is not None else None
    metrics = {k: ckpt.get(k) for k in ("val_f1", "val_loss", "val_precision", "val_recall") if k in ckpt}

    return ckpt, next_epoch, metrics, info


In [10]:
import inspect
from typing import Dict, Any, Optional
import torch
import torch.nn as nn

def _get_device(device: Optional[torch.device] = None) -> torch.device:
    if device is None:
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if isinstance(device, str):
        return torch.device(device)
    return device

def build_swinunetr(params: Dict[str, Any],
                           in_ch: int = 1,
                           out_ch: int = 1,
                           device: Optional[torch.device] = None,
                           wrap_dataparallel: bool = True) -> nn.Module:
    """
    Сборка SwinUNETR из словаря params:
    - фильтрует params по сигнатуре конструктора (чтобы игнорировать лишние ключи);
    - создаёт модель, при наличии >1 GPU оборачивает в DataParallel (если wrap_dataparallel=True);
    - возвращает модель на указанном device.
    """
    device = _get_device(device)
    sig = inspect.signature(SwinUNETR.__init__)
    allowed = {p for p in sig.parameters if p not in ("self", "in_channels", "out_channels")}
    call_kwargs = {k: v for k, v in params.items() if k in allowed}
    model = SwinUNETR(in_ch, out_ch, **call_kwargs)
    if wrap_dataparallel and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    return model.to(device)

MODEL_PARAMS = dict(
    patch_size=(2, 2, 2),
    depths=(2, 2, 2, 1),
    num_heads=(3, 6, 12, 24),
    window_size=(7, 7, 7),
    qkv_bias=True,
    mlp_ratio=4.0,
    feature_size=48,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    dropout_path_rate=0.1,
    patch_norm=True,
    spatial_dims=3,
)

In [11]:
import torch
import torch.nn as nn

class BinaryDiceLoss(nn.Module):
    def __init__(self, smooth: float = 1e-5):
        super().__init__()
        self.smooth = smooth

    def forward(self, logits: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        logits: Tensor with shape (B,1,D,H,W) or (B,D,H,W)  (raw logits)
        target: Tensor with shape (B,1,D,H,W) or (B,D,H,W)  (0/1 labels)
        """
        probs = torch.sigmoid(logits)

        # Если target без канального измерения — добавим
        if target.dim() == probs.dim() - 1:
            target = target.unsqueeze(1)

        # привести тип и устройство
        target = target.to(dtype=probs.dtype, device=probs.device)

        # выпрямляем по батчу
        probs_flat = probs.view(probs.size(0), -1)
        target_flat = target.view(target.size(0), -1)

        intersection = (probs_flat * target_flat).sum(dim=1)
        dice = (2.0 * intersection + self.smooth) / (probs_flat.sum(dim=1) + target_flat.sum(dim=1) + self.smooth)

        # средняя Dice loss по батчу
        return 1.0 - dice.mean()


class BinaryFocalLoss(nn.Module):
    def __init__(self, gamma: float = 2.0, alpha: float = 0.25, reduction: str = 'mean', eps: float = 1e-6):
        """
        alpha: weight for the positive class (class=1). Negative class weight will be (1-alpha).
        """
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction
        self.eps = eps

    def forward(self, logits: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        probs = torch.sigmoid(logits)

        if target.dim() == probs.dim() - 1:
            target = target.unsqueeze(1)

        # привести тип/устройство
        target = target.to(dtype=probs.dtype, device=probs.device)

        # boolean маска для класса 1
        pos_mask = target.eq(1.0)
        neg_mask = ~pos_mask

        # pt = p_t: вероятность правильного класса
        pt = torch.where(pos_mask, probs, 1.0 - probs)

        # защита от лог(0)
        pt = pt.clamp(min=self.eps, max=1.0 - self.eps)

        # альфа для каждого элемента: alpha для положительных, 1-alpha для отрицательных
        alpha_t = torch.where(pos_mask, self.alpha, 1.0 - self.alpha)

        focal_weight = (1.0 - pt) ** self.gamma

        # базовый лог-лист
        loss = - alpha_t * focal_weight * torch.log(pt)

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss  # 'none'


class CombinedLoss(nn.Module):
    def __init__(self, w_bce: float = 0.2, w_dice: float = 0.4, w_focal: float = 0.4):
        super().__init__()
        # BCEWithLogitsLoss принимает raw logits и target в формате float
        self.bce = nn.BCEWithLogitsLoss(reduction='mean')
        self.dice = BinaryDiceLoss()
        self.focal = BinaryFocalLoss(gamma=2.0, alpha=0.25, reduction='mean')
        self.w_bce = w_bce
        self.w_dice = w_dice
        self.w_focal = w_focal

    def forward(self, logits: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        # Приведём target по форме к logits, если нужно
        if target.dim() == logits.dim() - 1:
            target = target.unsqueeze(1)

        # привести тип и устройство
        target = target.to(dtype=logits.dtype, device=logits.device)

        loss_bce = self.bce(logits, target)
        loss_dice = self.dice(logits, target)
        loss_focal = self.focal(logits, target)

        return self.w_bce * loss_bce + self.w_dice * loss_dice + self.w_focal * loss_focal


In [12]:

import gc
# device info
ngpu = torch.cuda.device_count()
print("GPUs available:", ngpu)
if ngpu > 0:
    for i in range(ngpu):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

USE_DATAPARALLEL = True
MAIN_DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# create encoder and model on CPU first (safer) then move/wrap
# 2) Создаём модель и переносим на устройство
#model = AttentionFaultFormerNet(enc, num_classes=1, final_sigmoid=False)
model = build_swinunetr(MODEL_PARAMS, in_ch=1, out_ch=1)
model.to(MAIN_DEVICE)
# Move model to main device (cuda:0 if available)
ckpt_path = "/kaggle/working/checkpoint.pth"


for name, p in model.named_parameters():
    print("param", name, p.device)
for name, b in model.named_buffers():
    print("buffer", name, b.device)

# Wrap in DataParallel only if beneficial
if USE_DATAPARALLEL and ngpu > 1:
    device_ids = list(range(ngpu))
    print("Wrapping model in nn.DataParallel on device_ids =", device_ids)
    model.to('cuda:0')
    model = nn.DataParallel(model, device_ids=device_ids, output_device = 0)
    is_dp = True
else:
    if USE_DATAPARALLEL and ngpu <= 1:
        print("DataParallel requested but only <=1 GPU found — skipping DataParallel (use single-GPU).")
    is_dp = False

print("Model device (example param):", next(model.parameters()).device)
print("Is DataParallel:", isinstance(model, nn.DataParallel))

# utility: function to unwrap for saving / scheduler etc.
def unwrap_model(m):
    return m.module if isinstance(m, nn.DataParallel) else m

# used_params for checkpoint metadata
used_params = dict(embed_dim=48, use_checkpoint=True)

# Print parameter counts (optional)
total = sum(p.numel() for p in model.parameters())
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total params: {total/1e6:.3f} M, Trainable: {trainable/1e6:.3f} M")

BATCH_SIZE = 1


from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

# ---------------------------
# Prepare gdrive
# ---------------------------
gdrive_service, gdrive_folder_id = setup_gdrive_from_token(TOKEN_PATH, GDRIVE_FOLDER_NAME)
print("Drive configured?", gdrive_service is not None, gdrive_folder_id is not None)

# ---------------------------
# Loss, optimizer, scaler, scheduler
# ---------------------------

import torch.optim as optim

loss_fn = CombinedLoss()
optimizer = optim.AdamW(unwrap_model(model).parameters(), lr=2e-3)
scaler = torch.amp.GradScaler(enabled=torch.cuda.is_available())

SCHEDULER_TYPE = "ReduceLROnPlateau"
if SCHEDULER_TYPE == "ReduceLROnPlateau":
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=2)
elif SCHEDULER_TYPE == "CosineAnnealingLR":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
else:
    scheduler = None
model = model.to(MAIN_DEVICE)
ckpt, next_epoch, metrics, info = load_checkpoint(
    ckpt_path,
    model=model,           # модель уже перемещена на device / возможно DataParallel обёртка
    optimizer=optimizer,
    scheduler=scheduler,
    scaler=scaler,
    device=MAIN_DEVICE,
    load_optimizer=True,
    load_scheduler=True,
    load_scaler=True,
    strict=False,         # настраиваем, если архитектура чуть отличается
)
print(info)


GPUs available: 0
param swinViT.patch_embed.proj.weight cpu
param swinViT.patch_embed.proj.bias cpu
param swinViT.patch_embed.norm.weight cpu
param swinViT.patch_embed.norm.bias cpu
param swinViT.layers1.0.blocks.0.norm1.weight cpu
param swinViT.layers1.0.blocks.0.norm1.bias cpu
param swinViT.layers1.0.blocks.0.attn.relative_position_bias_table cpu
param swinViT.layers1.0.blocks.0.attn.qkv.weight cpu
param swinViT.layers1.0.blocks.0.attn.qkv.bias cpu
param swinViT.layers1.0.blocks.0.attn.proj.weight cpu
param swinViT.layers1.0.blocks.0.attn.proj.bias cpu
param swinViT.layers1.0.blocks.0.norm2.weight cpu
param swinViT.layers1.0.blocks.0.norm2.bias cpu
param swinViT.layers1.0.blocks.0.mlp.linear1.weight cpu
param swinViT.layers1.0.blocks.0.mlp.linear1.bias cpu
param swinViT.layers1.0.blocks.0.mlp.linear2.weight cpu
param swinViT.layers1.0.blocks.0.mlp.linear2.bias cpu
param swinViT.layers1.0.blocks.1.norm1.weight cpu
param swinViT.layers1.0.blocks.1.norm1.bias cpu
param swinViT.layers1.0

In [13]:
# Получаем главный девайс, который использует DataParallel (device_ids[0]) или MAIN_DEVICE
if isinstance(model, nn.DataParallel):
    dp_first = torch.device(f"cuda:{model.device_ids[0]}")
    # Переместим именно внутренний модуль на этот девайс
    model.module.to(dp_first)
    main_dev = dp_first
else:
    model.to(MAIN_DEVICE)
    main_dev = MAIN_DEVICE

# Проверка
for n,p in (model.module.named_parameters() if isinstance(model, nn.DataParallel) else model.named_parameters()):
    if p.device != main_dev:
        print("STILL WRONG:", n, p.device)

print("Model ready on", main_dev)


Model ready on cpu


In [ ]:
!mkdir "/content/checkpoints"

ROOT_DIR = "/content/checkpoints"

In [ ]:
from tqdm.notebook import tqdm
from datetime import datetime
import matplotlib.pyplot as plt

history = {"epochs": [], "train_loss": [], "val_loss": [], "val_f1": [], "val_precision": [], "val_recall": [], "lr": [], "saved_checkpoints": []}
best_val_f1 = -1
NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\n=== Epoch {epoch}/{NUM_EPOCHS} ===")
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    train_loss = train_epoch(model, train_loader, optimizer, loss_fn, epoch, scaler, MAIN_DEVICE, accumulation_steps=1)
    val_loss, val_f1, val_prec, val_rec = validate(model, val_loader, MAIN_DEVICE)
    current_lr = optimizer.param_groups[0]["lr"]

    print(
        f"Epoch {epoch} | Train loss: {train_loss:.4f} | Val loss: {val_loss:.4f} | "
        f"F1: {val_f1 if not np.isnan(val_f1) else 'nan'} | Prec: {val_prec if not np.isnan(val_prec) else 'nan'} | "
        f"Rec: {val_rec if not np.isnan(val_rec) else 'nan'} | LR: {current_lr:.2e}"
    )

    # === save checkpoint locally ===
    model_to_save = unwrap_model(model)
    ckpt_name = f"checkpoint_epoch_{epoch}.pth"
    ckpt_path_local = os.path.join(ROOT_DIR, ckpt_name)
    checkpoint = {
        "epoch": epoch,
        "model_state_dict": model_to_save.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "val_f1": val_f1,
        "val_loss": val_loss,
        "val_precision": val_prec,
        "val_recall": val_rec,
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "used_model_params": used_params,
    }
    torch.save(checkpoint, ckpt_path_local)
    print(f"[save] Saved checkpoint locally: {ckpt_path_local} (size={os.path.getsize(ckpt_path_local)} bytes)")

    # update best
    if (not np.isnan(val_f1) and val_f1 > best_val_f1):
        best_val_f1 = val_f1
        best_path_local = os.path.join(ROOT_DIR, "best_checkpoint.pth")
        torch.save(checkpoint, best_path_local)
        print(f"[save] Updated best checkpoint: {best_path_local} (size={os.path.getsize(best_path_local)} bytes)")

    # update history
    history["epochs"].append(epoch)
    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["val_f1"].append(val_f1)
    history["val_precision"].append(val_prec)
    history["val_recall"].append(val_rec)
    history["lr"].append(current_lr)
    history["saved_checkpoints"].append(ckpt_path_local)

    # save history JSON
    history_path = os.path.join(ROOT_DIR, "training_history.json")
    try:
        with open(history_path, "w", encoding="utf-8") as fh:
            json.dump(history, fh, ensure_ascii=False, indent=2)
        print(f"[save] Saved history JSON: {history_path} (size={os.path.getsize(history_path)} bytes)")
    except Exception as e:
        print("[save] Failed to save training_history.json:", e)

    # save loss plot
    loss_plot_path = os.path.join(ROOT_DIR, "training_loss.png")
    try:
        plt.figure(figsize=(8,5))
        epochs_range = list(range(1, len(history["train_loss"]) + 1))
        plt.plot(epochs_range, history["train_loss"], label="Train Loss")
        plt.plot(epochs_range, history["val_loss"], label="Val Loss")
        plt.title("Training and Validation Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(loss_plot_path)
        plt.close()
        print(f"[save] Saved loss plot: {loss_plot_path} (size={os.path.getsize(loss_plot_path)} bytes)")
    except Exception as e:
        loss_plot_path = None
        print("[save] Failed to plot/save loss figure:", e)

    # prepare upload list
    to_upload = [ckpt_path_local, history_path]
    if loss_plot_path:
        to_upload.append(loss_plot_path)
    if 'best_path_local' in locals():
        to_upload.append(best_path_local)

    # === Upload to Google Drive (each epoch) with diagnostics ===
    print(f"[upload] Drive configured? service={gdrive_service is not None}, folder_id={gdrive_folder_id is not None}")
    uploaded_any = False
    if gdrive_service is not None and gdrive_folder_id is not None:
        for p in to_upload:
            ok = try_upload_file(gdrive_service, gdrive_folder_id, p)
            uploaded_any = uploaded_any or ok
        if uploaded_any:
            print("[upload] One or more files uploaded successfully.")
        else:
            print("[upload] No files were uploaded (see messages above).")
        print("Drive folder link:", f"https://drive.google.com/drive/folders/{gdrive_folder_id}")
    else:
        print("[upload] Skipping upload: Drive not configured. Place token.json and rerun setup_gdrive_from_token().")

    # scheduler step
    if scheduler is not None:
        try:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()
        except Exception as e:
            print("Scheduler step failed:", e)

print("Training finished. Best val F1:", best_val_f1)


=== Epoch 1/20 ===


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Train E1:   0%|          | 0/1596 [00:00<?, ?it/s]